# Introduction to jupyter lab & dusk/dawn

In this notebook, we want to give you an introduction to jupyter lab & dusk/dawn. The goal is to learn all prerequisites so that you can solve the exercises. We start with jupyter lab:

### Jupyter Lab

Jupyter Lab gives us a convenient environment to work with notebooks. The current web interface is Jupyter Lab. It has some very nice features that we want to go through:

#### File Explorer

On the left hand side you will find a simple file explorer with the following useful functionality.

You can navigate between folders by double clicking folders. At the top, you will see the current path and you can navigate back to the starting location.

Double clicking a file will open it. Depending on the file type it will be opened differently. Jupyter Lab also has a basic text editor, so you can open textual files.

It is likely that you opened jupyter lab through mybinder or by running a docker image. In that case, it's useful to download files onto your local machine. You can do that by right clicking a file and then selecting download. Vice versa, you can also upload files by clicking the upload file button (up arrow icon) at the top button toolbar of the file explorer.

The file explorer is quite useful and you're generally encouraged to look for features yourselves.

#### Launcher

The next feature we want to look at is the launcher. It is accessible through the file explorere. The plus icon at the top toolbar of the file explorer will open the launcher. With the launcher you can start new python notebooks, python consoles and even a basic terminal. This way you can, e.g., experiment with dusk & dawn more interactively by using a python console or a terminal. Additionally, you can create your own python notebooks for experimentation and if you want, you can also download it.

#### Windowing

Jupyter lab has pretty good support for having different _windows_ open. If you open multiple notebooks, terminals or consoles, they will all be accessible from the top bar. Almost at the top of this web interface. By dragging the window title, you can arrange the windows however you want. E.g., side by side or top/down arrangements are possible as well. You are also encouraged to arrange them to your liking. Some useful layouts could be to open an exercise notebook side by side with the corresponding exercise slides or to have a python console below your exercise notebook, to experiment with dusk/dawn's functionality while solving the exercise.

### Jupyter Lab's notebooks

What you are currently reading is called a python notebook. Here we want to give you a quick introduction to that.

A notebook consists of cells. These cells can contain different types of content. E.g., markdown text (as is the case for this cell) or code.
Below is an example of a cell with code:

In [1]:
# This is a cell with python code.
print("Hello, World!")

# We can write python code which will be executed and we'll see the output of that code below:

Hello, World!


The output of a cell always corresponds to an execution of that cell. If the code of the cell gets modified, we'll only see the modifications in the output if we execute the cell again. At the top of this notebook, you find a toolbar with various actions we can carry out on cells. This toolbar includes the possibility to execute a cell (the _play_ button). Try it for the cell below:

In [ ]:
print("".join(map(lambda c: chr(ord(c) - 2), "Kv\"yqtmgf#")))

You should see as the output "It worked!".

You can add new cells by clicking the plus button in the _cell's toolbar_. The type of the cell can be changed with the drop down menu in the same toolbar. Code cells are probably the most useful for you.

Sometimes it can take some time for a cell to execute. While the cell executes, you will see a star ('[\*]') on its left hand side. Try it with the cell below:

In [ ]:
from time import sleep

for word in "While the cell executes, you will see a star ('[*]') on its left hand side.".split():
    print(word + " ", end="")
    sleep(0.8)
print()

Feel free to experiment with the top toolbar a bit.

The order of executing the cell matters, since each cell can introduce and use variables for other cells. Sometimes, your code doesn't work anymore, because the cells were executed in a bad order. You can reset this easily, by clicking the button to restart and execute the whole notebook again (the two left arrows). This is somewhat similar to `make clean` when your build folder got into a strange state.

What's also useful is that you can execute terminal commands in cells by prefixing them with a '!'. Here's an example:

In [4]:
!echo these are all bash commands below that work
# e.g., we can print the version of gcc:
!echo
!echo GCC\'s version:
!gcc --version
# or the version of clang-format
!echo clang-format\'s version:
!clang-format --version

these are all bash commands below that work

GCC's version:
gcc (Ubuntu 9.3.0-10ubuntu2) 9.3.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

clang-format's version:
clang-format version 10.0.0-4ubuntu1 


That's it for the notebooks. If you want to learn more or aren't sure about something, you can also use the [jupyter lab documentation](https://jupyter.org/documentation).

### dusk/dawn usage in notebooks:

Here we'll have a look at how to use dusk & dawn inside noteboosk. Since we can write python code inside jupyter notebooks and dusk is a python embedded DSL, we can naturally write our dusk stencils inside the notebooks. Let's look at a simple example below:

In [5]:
from dusk.script import *

@stencil
def simple_example_stencil(
    a: Field[Edge]
):
    with levels_upward:
        a = 1

Next we can transpile this stencil to SIR with dusk's python API:

In [6]:
from dusk.transpile import callable_to_pyast, pyast_to_sir

sir = pyast_to_sir(callable_to_pyast(
    # note that here we can use the name of the stencil
    # (technically, this is the name of the python function which is marked as a dusk stencil)
    simple_example_stencil
))

`sir` now contains the stencil in SIR form. Let's see what this looks like:

In [7]:
sir

gridType: Unstructured
filename: "<unknown>"
stencils {
  ast {
    root {
      block_stmt {
        statements {
          vertical_region_decl_stmt {
            vertical_region {
              ast {
                root {
                  block_stmt {
                    statements {
                      expr_stmt {
                        expr {
                          assignment_expr {
                            left {
                              field_access_expr {
                                name: "a"
                                unstructured_offset {
                                }
                              }
                            }
                            op: "="
                            right {
                              literal_access_expr {
                                value: "1"
                                type {
                                  type_id: Integer
                                }
                              }
 

This is an _abstract syntax tree_. For our purposes, the content of this is not that interesting. However, this is what we can feed into dawn to generate C++ code. Dusk can do that for us by calling into dawn (through its python bindings `dawn4py`). Unfortunately, there is a bug when C++ exceptions get raised inside of python notebooks (such as this one) which leads to error messages disappearing. For the sake of debugging (which we probably will need when solving the exercises), we will instead write the SIR to file and then use `dawn-opt` and `dawn-codegen` directly:

In [8]:
from dusk.transpile import sir_to_json

with open("intro.sir", "w") as f:
    f.write(sir_to_json(sir))

!dawn-opt intro.sir | dawn-codegen -b naive-ico -o intro_cxx-naive.cpp

Now we can have a look at the generated C++ code. It's probably better to use `clang-format` to get a more readable version of it. _Readable_ is probably not the correct word here. Please bare with us, readability can be improved in the future, but hasn't been a strong focus so far.

In [9]:
!clang-format -i intro_cxx-naive.cpp
!cat intro_cxx-naive.cpp

#define DAWN_GENERATED 1
#undef DAWN_BACKEND_T
#define DAWN_BACKEND_T CXXNAIVEICO
#define GRIDTOOLS_DAWN_NO_INCLUDE
#include <driver-includes/math.hpp>
#include <driver-includes/unstructured_domain.hpp>
#include <driver-includes/unstructured_interface.hpp>

namespace dawn_generated {
namespace cxxnaiveico {
template <typename LibTag> class simple_example_stencil {
private:
  struct stencil_18 {
    ::dawn::mesh_t<LibTag> const &m_mesh;
    int m_k_size;
    ::dawn::edge_field_t<LibTag, ::dawn::float_type> &m_a;
    ::dawn::unstructured_domain m_unstructured_domain;

  public:
    stencil_18(::dawn::mesh_t<LibTag> const &mesh, int k_size,
               ::dawn::edge_field_t<LibTag, ::dawn::float_type> &a)
        : m_mesh(mesh), m_k_size(k_size), m_a(a) {}

    ~stencil_18() {}

    void sync_storages() {}
    static constexpr ::dawn::driver::unstructured_extent a_extent = {false, 0,
                                                                     0};

    void run() {
      using :

Now that we got our C++ code from the dusk stencil, we'll want to run it. For this we'll need some driver code. We've prepared the driver code for you already in the file `driver.cpp`. To build it, we can use the already prepared `Makefile`:

In [10]:
!make

This creates an executable `runner` which we can use to run:

In [11]:
!./runner

As expected, `a` == 1 everywhere.


The runner contains a small check function to verify that `a` correctly has the value `1` everywhere. The output above should correspond to:

    As expected, `a` == 1 everywhere.

All of our exercises follow this structure of having a `driver.cpp` and a `Makefile` that produce a `runner` executable.

### dusk/dawn trouble shooting & errors

To help you solve the exercises, we want to show you various errors you might encouter and how to trouble shoot them.

#### Python syntax errors

We'll start with Python syntax errors. Before dusk can process the stencil, it has to be syntacticaly valid python code. Below is an example of a python syntax error. Note the output which indicates that it indeed was a python syntax error:

In [12]:
@stencil
def a_stencil_with_a_python_syntax_error(a: Field[Edge]):
    with levels_upward:
        # in this expression there is a missing bracket:
        a = (a + 1

SyntaxError: unexpected EOF while parsing (<ipython-input-12-9c8f01f4a5c3>, line 5)

#### dusk errors

The next phase that can fail is the translation from the dusk stencil to SIR. In the following example, we are using an undefined variable:

In [13]:
@stencil
def a_stencil_with_a_dusk_error(a: Field[Edge]):
    with levels_upward:
        # This code uses an undefined variable.
        # This isn't an error in python due to its dynamic nature.
        # However, in dusk this is statically checked
        a = undefined_variable + 1

We didn't get an error yet? That's correct. We first have to start the translation to SIR. Note that this error message is very long, since it contains the full stack-trace until the error. It's usually best to scroll down all the way and see what's the message of the error. We tried to use _descriptive_ error message.

In [14]:
sir = pyast_to_sir(callable_to_pyast(a_stencil_with_a_dusk_error))

DuskSyntaxError: DuskSyntaxError: Undeclared variable 'undefined_variable'!
at None
(undefined_variable)

Even though the error message is very long, we get a useful message in the end:

    DuskSyntaxError: Undeclared variable 'undefined_variable'!

#### Dawn errors

Dawn errors can occur when we invoke `dawn-opt` or `dawn-codegen`. Dawn will carry out many checks based on the _locational information_ provided by the various declarations.

In this example below we try to write a cell field to an edge field which doesn't make that much sense, since there's a lot more edges than cells in a triangular mesh:

In [15]:
@stencil
def invalid_field_assignment(
    edge_field: Field[Edge],
    cell_field: Field[Cell],
):
    with levels_upward:
        edge_field = cell_field

In [16]:
with open("intro.sir", "w") as f:
    f.write(sir_to_json(pyast_to_sir(callable_to_pyast(invalid_field_assignment))))

!dawn-opt intro.sir | dawn-codegen -b naive-ico -o intro_cxx-naive.cpp

Assertion failed: `dimsConsistent' Dimensions consistency check failed at line -1 
Function: 'bool dawn::PassValidation::run(const std::shared_ptr<dawn::iir::StencilInstantiation>&, const dawn::Options&, const string&)'
Location: ../dawn/src/dawn/Optimizer/PassValidation.cpp:47
Aborted (core dumped)
terminate called after throwing an instance of 'std::runtime_error'
  what():  Cannot deserialize input
Aborted (core dumped)


Given our declarations, dawn can check this case and throw an error. (Error messages aren't high quality due to the current prototyping state)

#### C++ errors

C++ errors will be triggered when we invoke `make`. This can happen if the signature of the stencil doesn't match what the driver expects. So take care not to change the function signature of the dusk stencil:

In [17]:
@stencil
def simple_example_stencil(
    a: Field[Edge],
    # The parameter below changes the signature and thus it becomes incompatible with the driver
    b: Field[Cell]
):
    with levels_upward:
        a = 1

with open("intro.sir", "w") as f:
    f.write(sir_to_json(pyast_to_sir(callable_to_pyast(simple_example_stencil))))

!dawn-opt intro.sir | dawn-codegen -b naive-ico -o intro_cxx-naive.cpp

!make

driver.cpp: In function ‘int main(int, char**)’:
driver.cpp:170:27: error: no matching function for call to ‘dawn_generated::cxxnaiveico::simple_example_stencil<atlasInterface::atlasTag>::simple_example_stencil(atlas::Mesh&, const int&, std::tuple_element<0, std::tuple<atlasInterface::Field<double> > >::type&)’
  170 |       mesh, k_size, a_view)
      |                           ^
In file included from driver.cpp:29:
intro_cxx-naive.cpp:53:3: note: candidate: ‘dawn_generated::cxxnaiveico::simple_example_stencil<LibTag>::simple_example_stencil(dawn::mesh_t<LibTag>&, int, dawn::edge_field_t<LibTag, double>&, dawn::cell_field_t<LibTag, double>&) [with LibTag = atlasInterface::atlasTag; dawn::mesh_t<LibTag> = atlas::Mesh; dawn::edge_field_t<LibTag, double> = atlasInterface::Field<double>; dawn::cell_field_t<LibTag, double> = atlasInterface::Field<double>]’
   53 |   simple_example_stencil(const ::dawn::mesh_t<LibTag> &mesh, int k_size, ::dawn::edge_field_t<LibTag, ::dawn::float_type>& a, 

Also take care that you don't change the name of the stencil, since that's part of the signature:

In [18]:
@stencil
def different_name(a: Field[Edge]):
    with levels_upward:
        a = 1

with open("intro.sir", "w") as f:
    f.write(sir_to_json(pyast_to_sir(callable_to_pyast(simple_example_stencil))))

!dawn-opt intro.sir | dawn-codegen -b naive-ico -o intro_cxx-naive.cpp

!make

driver.cpp: In function ‘int main(int, char**)’:
driver.cpp:170:27: error: no matching function for call to ‘dawn_generated::cxxnaiveico::simple_example_stencil<atlasInterface::atlasTag>::simple_example_stencil(atlas::Mesh&, const int&, std::tuple_element<0, std::tuple<atlasInterface::Field<double> > >::type&)’
  170 |       mesh, k_size, a_view)
      |                           ^
In file included from driver.cpp:29:
intro_cxx-naive.cpp:53:3: note: candidate: ‘dawn_generated::cxxnaiveico::simple_example_stencil<LibTag>::simple_example_stencil(dawn::mesh_t<LibTag>&, int, dawn::edge_field_t<LibTag, double>&, dawn::cell_field_t<LibTag, double>&) [with LibTag = atlasInterface::atlasTag; dawn::mesh_t<LibTag> = atlas::Mesh; dawn::edge_field_t<LibTag, double> = atlasInterface::Field<double>; dawn::cell_field_t<LibTag, double> = atlasInterface::Field<double>]’
   53 |   simple_example_stencil(const ::dawn::mesh_t<LibTag> &mesh, int k_size, ::dawn::edge_field_t<LibTag, ::dawn::float_type>& a, 

#### Runner errors

Lastly, we'll show an error that can occur when running a stencil through the `runner`. The runner will crash, e.g., when a floating point exception occurs. This is enabled in this particular driver (other drivers can turn this off). So if a division by zero occurs, the runner will crash

In [19]:
@stencil
def simple_example_stencil(a: Field[Edge]):
    with levels_upward:
        # a trivial division by zero that will cause the runner to crash
        a = 1/0


with open("intro.sir", "w") as f:
    f.write(sir_to_json(pyast_to_sir(callable_to_pyast(simple_example_stencil))))

!dawn-opt intro.sir | dawn-codegen -b naive-ico -o intro_cxx-naive.cpp
!make
!./runner

Floating point exception (core dumped)


That's it for the trouble shooting part. If you face issues or have questions, please write in the dedicated slack.

### Dusk syntax & illegal language elements

When solving the exercises, you will probably find that dusk is very restrictive in what it allows inside its stencil bodies. This is by design for the following two reason:

1) If we are too permissive early on, we might accidentally allow things that cause issues later on. We chose the route to be more restrictive initially and when we want to allow new things, we can review whether they could cause issues. This is a more controlled approach.

2) We can avoid backwards-compatibility breaking changes more easily.

Basically, dusk whitelists certain elements one by one and everything else is illegal. This usually leads to matcher errors and sometimes semantic errors. As a result, most python language elements aren't allowed:

In [20]:
@stencil
def no_containment_check(a: Field[Edge]):
    with levels_upward:
        a = 1 if 1 in [1, 2] else 0

pyast_to_sir(callable_to_pyast(no_containment_check))

DuskSyntaxError: DuskSyntaxError: Unsupported comparison operator '<_ast.In object at 0x7f310905d580>'!
at None
(<_ast.In object at 0x7f310905d580>)

In [21]:
@stencil
def no_list_comprehension(a: Field[Edge]):
    with levels_upward:
        a = [2*x for x in range(5)]

pyast_to_sir(callable_to_pyast(no_list_comprehension))

DuskSyntaxError: DuskSyntaxError: Unrecognized node: '<_ast.ListComp object at 0x7f30fc11b1c0>'!
at Line: 4 - 4; Col: 12 - 35
(<_ast.ListComp object at 0x7f30fc11b1c0>)

If you encounter such an error, you probably have to rewrite your code to use the allowed language elements. If you believe your code should be allowed, feel free to write us in slack. The idea is to also allow other elements based on user feedback!

#### Python positional vs keyword arguments

Here we want to quickly introduce python's positional arguments and keyword arguments. If you're already familiar with them, feel free to skip it.

Python functions can have two kinds of arguments:

1) Positional arguments

2) Keyword arguments

(technically, we could also distinguish between _positional only_, etc arguments, but let's keep it simple for now)

Here's an example how to declare and use them:

In [22]:
# a python function with positional arguments
def positional(a, b, c):
    ...

# with positional arguments, only the position matters
positional(3, 4, 5) # will have a = 3, b = 4, c = 5

# but an argument has to be given for each parameter
# positional(3, 4) # this will crash, c is missing!
    
# a python function with keyword arguments
# note that the assignment syntax is used for those (`=`)
def keyword(a = 1, b = 2, c = 3):
    ...

# with keyword arguments, we can use the assignment syntax when calling the function:
keyword(a = 10, b = 11, c = 12) # will have a = 10, b = 11, c = 12

# additionally, keyword arguments can be skipped
# (this is only true if a default value is given. In dusk, all keyword arguments have default values)
keyword() # will have a = 1, b = 2, c = 3
# we can also only provide a subset of all keyword arguments:
keyword(a = 100, c = 101) # will have a = 100, b = 2, c = 101

# a python function with both positional & keyword arguments:
def mixed(a, b, c = 3, d = 4):
    ...

# the positional arguments are mandatory, whereas the keyword arguments are optional:
mixed(6, 7, c = 8, d = 9) # will have a = 6, b = 7, c = 8, d = 9
# we can skip the keyword arguments:
mixed(6, 7) # will have a = 6, b = 7, c = 3, d = 4

# however, all positional arguments must be provided:
# mixed(c = 10, d = 11) # will crash, positional arguments a and b are not given!

That's it for the introduction to the exercise tools. Please contact us in slack if you encounter issues.
Good luck and have fun with the exercises!